#  YaProfi baseline

In [82]:
!rm -rf kapture

In [83]:
USER = !pwd
USER = USER[0]
print(USER)

/home/yap202110


In [84]:
!mkdir kapture
from tqdm import tqdm
import os, shutil
def copytree(src, dst, symlinks=False, ignore=None):
    for item in tqdm(os.listdir(src)):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
copytree('{}/YaProfi'.format(USER),'{}/kapture'.format(USER))


100%|██████████| 2/2 [11:35<00:00, 347.63s/it]


In [85]:
!unzip $USER/model/Resnet101-AP-GeM-LM18.pt.zip
!mv Resnet101-AP-GeM-LM18.pt /home/repo/deep-image-retrieval

Archive:  /home/yap202110/model/Resnet101-AP-GeM-LM18.pt.zip
  inflating: Resnet101-AP-GeM-LM18.pt  


In [95]:
import os
from tqdm import tqdm

class Structure():
    
    '''
    Make appropriate for Kapture structure :
    
    kapture_dir/
        query/
            sensors/
                ...
        mapping/
            sensors/
                ...
    '''

    def __init__(self, root):
   
        self.root = root
        self.frequency = {'query': 1, 'mapping': 20}
        self.dates = {'mapping':['2021-03-27-09-08-15','2021-03-27-10-17-01', '2021-03-27-10-32-53'],'query':['2021-03-27-03-30-47']}

    def mkstructure(self):
        
        !mkdir $self.root/query/sensors
        !mkdir $self.root/mapping/sensors
        !mkdir $self.root/query/sensors/records_data
        !mkdir $self.root/mapping/sensors/records_data
        modes = ['query','mapping']
        for mode in modes:
            freq = self.frequency[mode]
            records_camera = open(f'{self.root}/{mode}/sensors/records_camera.txt', "w")
            records_camera.write('# timestamp, device_id, image_path\n')
            
            sensor =  open(f'{self.root}/{mode}/sensors/sensors.txt', 'w')
            sensor.write('# sensor_id, name, sensor_type, [sensor_params]+\n')
            
            trajectories = open(f'{self.root}/{mode}/sensors/trajectories.txt','w')
            trajectories.write('# timestamp, device_id, tx, ty, tz, qw, qx, qy, qz\n')
            
            
            dates = sorted(self.dates[mode])
            count = 0  ###################### !! true timestaps will be better
            for date in dates:
                images = sorted(os.listdir(f'{self.root}/{mode}/{date}/rgb_left'))
                !cp $self.root/$mode/$date/rgb_left/* $self.root/$mode/sensors/records_data
                for index,image in tqdm(enumerate(images)):
                    if index % freq == 0:
                        string_records_camera = f'{count}, sensor{count}, {self.root}/{mode}/sensors/records_data//{image}' +'\n'
                        records_camera.write(string_records_camera)

                        string_sensor = f'sensor{count}, , camera, PINHOLE, 1280, 720, 684.06805, 684.06805, 615.11951 , 345.32159 \n'
                        sensor.write(string_sensor)
                        count+=1 
                    
 

            records_camera.close()
            sensor.close()
        
        count = 0 
        gt_train =  sorted(list(map(lambda x: '{}/mapping/{}/gt_tum.txt'.format(self.root,x) ,self.dates['mapping'])))
        for file_path in gt_train:
            with open(file_path,'r') as f:
                for string in f.readlines():
                    timestamp, tx, ty, tz, qw, qx, qy, qz = string.split(' ')
                    qz = qz.split('\n')[0]
                    trajectories.write(f'{count}, sensor, {tx}, {ty}, {tz}, {qw}, {qx}, {qy}, {qz}' +'\n')
                    count+=1
        trajectories.close()


In [96]:
path = f'{USER}/kapture'
structure = Structure(path)
structure.mkstructure()
for_rm = filter(lambda x: x != 'sensors',os.listdir(path+'/query'))

mkdir: cannot create directory ‘/home/yap202110/kapture/query/sensors’: File exists
mkdir: cannot create directory ‘/home/yap202110/kapture/mapping/sensors’: File exists
mkdir: cannot create directory ‘/home/yap202110/kapture/query/sensors/records_data’: File exists
mkdir: cannot create directory ‘/home/yap202110/kapture/mapping/sensors/records_data’: File exists


13424it [00:00, 499457.44it/s]
15292it [00:00, 1291958.84it/s]
9325it [00:00, 1366927.09it/s]
3702it [00:00, 1029526.15it/s]


/bin/sh: 1: fm: not found
/bin/sh: 1: fm: not found
/bin/sh: 1: fm: not found
/bin/sh: 1: fm: not found
/bin/sh: 1: fm: not found
/bin/sh: 1: fm: not found
/bin/sh: 1: fm: not found
/bin/sh: 1: fm: not found


In [100]:
class FeatureExctractor():
  
  def __init__(self, root):
    self.root = root
    self.modes = ['query', 'mapping']

  def extract_global_features(self):
    for mode in self.modes:
        kapture_root = USER + '/kapture/{}'.format(mode)
        print(kapture_root)
        %cd $self.root/deep-image-retrieval
        !python3 -m dirtorch.extract_kapture --kapture-root $kapture_root --checkpoint Resnet101-AP-GeM-LM18.pt --gpu 0
        %cd $USER/kapture

  def extract_local_features(self, TOP_K = 5000, min_size = 225, min_scale = 0.3, max_scale = 1.0):
    for mode in self.modes:
        kapture_root = USER + '/kapture/{}'.format(mode)
        !python3 $self.root/r2d2/extract_kapture.py --kapture-root $kapture_root --model $self.root/r2d2/models/r2d2_WASF_N8_big.pt 




In [98]:
path = '/home/repo'
extractor = FeatureExctractor(path)
extractor.extract_global_features()
extractor.extract_local_features()

/home/yap202110/kapture/query
/home/repo/deep-image-retrieval
Launching on GPUs 0
=> loading checkpoint 'Resnet101-AP-GeM-LM18.pt' (current_iter 376)
loading /home/yap202110/kapture/query

Evaluation on Dataset: ImageList
  13424 images
  root: /home/yap202110/kapture/query/sensors/records_data...
DB: 100%|█████████████████████████████████| 13424/13424 [08:47<00:00, 25.43it/s]
writing extracted global features
100%|██████████████████████████████████| 13424/13424 [00:00<00:00, 16894.31it/s]
Features extracted.
/home/yap202110/kapture
/home/yap202110/kapture/mapping
/home/repo/deep-image-retrieval
Launching on GPUs 0
=> loading checkpoint 'Resnet101-AP-GeM-LM18.pt' (current_iter 376)
loading /home/yap202110/kapture/mapping

Evaluation on Dataset: ImageList
  1418 images
  root: /home/yap202110/kapture/mapping/sensors/records_data...
DB: 100%|███████████████████████████████████| 1418/1418 [00:56<00:00, 25.18it/s]
writing extracted global features
100%|████████████████████████████████████|

In [101]:
from kapture.io.features import keypoints_to_filepaths, image_keypoints_from_file
from kapture.io.records import images_to_filepaths
from kapture.io.image import image_keypoints_to_image_file
import kapture.io.csv as csv

kapture_data = csv.kapture_from_dir(f'{USER}/kapture/query')
kapture_data

sensors        : 13424
trajectories   :    0
records_camera : 13424
global_features : 13424

## RESULTS

In [102]:
mapping = f'{USER}/kapture/mapping' 
query = f'{USER}/kapture/query' 
!python3 /home/repo/kapture-localization/tools/kapture_compute_image_pairs.py --query $query --mapping $mapping -o pairs_mapping_3.txt --topk 3 

In [104]:
imageslist = []
traject_list = []

with open('{}/kapture/mapping/sensors/records_camera.txt'.format(USER)) as fin:
  for j in fin.readlines():
    imageslist.append(j.split(', ')[-1].split('\n')[0])


with open('{}/kapture/mapping/sensors/trajectories.txt'.format(USER)) as fin:
  for j in fin.readlines()[1:]:
    traject_list.append(tuple(j.split(', ')[2:]))


image_to_trajectories = {}
for key, value in zip(imageslist, traject_list):
  image_to_trajectories[key] = value

image_to_trajectories

{'image_path': ('0.0625332807773',
  '0.0519735212683',
  '-0.131324921913',
  '-0.494339365987',
  '0.498076820036',
  '-0.497769463329',
  '0.509679932855\n'),
 '/home/yap202110/kapture/mapping/sensors/records_data//2021-03-27-09-08-15_left_000000.png': ('0.0625332699133',
  '0.0519735548747',
  '-0.131324920322',
  '-0.494339366759',
  '0.498076819205',
  '-0.497769462541',
  '0.509679933689\n'),
 '/home/yap202110/kapture/mapping/sensors/records_data//2021-03-27-09-08-15_left_000020.png': ('0.0799371158376',
  '0.0748268204726',
  '-0.114755366868',
  '-0.49372337921',
  '0.499098036561',
  '-0.497963965634',
  '0.509087677765\n'),
 '/home/yap202110/kapture/mapping/sensors/records_data//2021-03-27-09-08-15_left_000040.png': ('0.0799371158376',
  '0.0748268204726',
  '-0.114755366868',
  '-0.49372337921',
  '0.499098036561',
  '-0.497963965634',
  '0.509087677765\n'),
 '/home/yap202110/kapture/mapping/sensors/records_data//2021-03-27-09-08-15_left_000060.png': ('0.0708658075144',
  '

In [ ]:
list_matches_Ap_Gem = []
answer_loc = []
with open('pairs_mapping_3.txt','r') as f:
  for line in f:
    list_matches_Ap_Gem.append(line)
    
list_matches_Ap_Gem = list_matches_Ap_Gem[2:]

s = slice(0, len(sorted(list_matches_Ap_Gem,key = lambda x: x[0])),3)
indices = range(s.start, s.stop, s.step)
it = iter(indices)


for pair in it:
  ap_gem = sorted(list_matches_Ap_Gem,key = lambda x: x[0])[pair:pair+3]
  ap_gem_query, ap_gem_mapping1, ap_gem_metric1 = ap_gem[0].split(', ')
  answer_loc.append(image_to_trajectories[ap_gem_mapping1])

In [ ]:
%cd USER
answer = list(map(lambda x:x[:3], answer_loc))
submission = open('submission.txt','w')
for frame in answer:
    string = ' '.join(frame)
    submission.write(string+'\n')
    
submission.close()

In [65]:
!pwd

/home/repo/deep-image-retrieval
